In [ ]:
#Implement CBC mode
#importing classes
import random
from random import randint
import os
import string
# The below file is given by OPEN SSL
!pip install pycrypto
from Crypto.Cipher import AES
from math import ceil
from google.colab import files
from binascii import unhexlify, b2a_base64, a2b_base64, hexlify
import chardet
import io


     |████████████████████████████████| 450kB 6.2MB/s 
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp37-cp37m-linux_x86_64.whl size=499931 sha256=8be49cf66d85c3d69e160f5db5ae39a3f98044e36e20e31712808e559a57f924
  Stored in directory: /root/.cache/pip/wheels/27/02/5e/77a69d0c16bb63c6ed32f5386f33a2809c94bd5414a2f6c196
Successfully built pycrypto


In [ ]:
uploadedmessage = files.upload()

Saving data.txt to data.txt


#Steps are
# 1) Padding of bits make it multiple of block size (used same function pkc7)
# 2) Xor the message with the IV
# 3) Use AES encyption to encrypt
# 4) For all the blocks  of message repeat steps from 2-4 except now use cipher text irrespective of IV

In [ ]:
def pkcs7(message, blocksize):
  padlen = blocksize-len(message)%blocksize
  if(padlen == blocksize):
    padlen = 0
  pkcsmessage=bytes([padlen])*padlen
  return message+pkcsmessage

In [ ]:
# Helper classes
# ECB AES-128 encyption
def aes_128_enc(message,key):
  ciphertext=AES.new(key,AES.MODE_ECB)
  return ciphertext.encrypt(message)

# ECB AES-128 decyption
def aes_128_dec(ciphertext,key):
  plaintext=AES.new(key,AES.MODE_ECB)
  return plaintext.decrypt(ciphertext)

# Xor of blocks of plain text and cipher text
def xor_cipher_plaintext(plaintext,ciphertext):
  xored=[]
  for p,c in zip(plaintext,ciphertext):
    xored.append(p^c)
  return bytes(xored)

#convert Bytes to string 
def bytes_to_str(bytemessage):
  return "".join(filter(lambda x: x in string.printable, "".join(map(chr, bytemessage))))





In [ ]:
def implement_CBC_ENCRYPTION(message,key):
  #Get blocks of messages
  byte_message=bytes(message,'utf-8')
  IV=b'\x00'*16
  next_ciphertext=IV
  ciphertext=b''
  blocksize=16
  for blocks in range(0,len(byte_message),16):
    #XOR the plaintext and cipher text
    paddedmessage=pkcs7(byte_message[blocks:blocks+16],len(key))
    xor_value=xor_cipher_plaintext(paddedmessage,next_ciphertext)
    next_ciphertext=aes_128_enc(xor_value,key)
    ciphertext+=next_ciphertext
  return ciphertext


In [ ]:
def implement_CBC_DECRYPTION(ciphertext,key):
  #Get blocks of messages
  byte_message=ciphertext
  IV=b'\x00'*16
  next_plaintext=IV
  plaintext=b''
  blocksize=16
  for blocks in range(0,len(byte_message),len(key)):
    #XOR the plaintext and cipher text
    paddedmessage=pkcs7(byte_message[blocks:blocks+len(key)],len(key))
    paddedmessage=aes_128_dec(paddedmessage,key)
    xor_value=xor_cipher_plaintext(paddedmessage,next_plaintext)
    plaintext+=xor_value
    next_plaintext=byte_message[blocks:blocks+len(key)]
  return plaintext

In [ ]:
if __name__== "__main__":
  text=""
  for key in uploadedmessage:
    text=uploadedmessage[key]
    break
  text=io.BytesIO(text)
  byte_string = b''.join(a2b_base64(line.strip()) for line in text.readlines())
  decryptedmessage=implement_CBC_DECRYPTION(byte_string, b'YELLOW SUBMARINE')
  decryptedmessage=bytes_to_str(decryptedmessage)
  for line in decryptedmessage.split("\n")[:10]:
    print(line)


I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
